## Web Scrapping Project
Objective: Utilize a list of addresses to locate the corresponding Redfin web pages and retrieve essential information pertaining to the apartment's market status and estimated price. This scalable approach can be applied to real estate operations, ensuring that the status of every apartment of interest remains up-to-date.
The advantage of this approach is that it eliminates the need for Selenium, offering a cost-effective and low-risk methodology that relies on Google Search, Request, and Beautiful Soup. With this method, I have not encountered any issues related to potential page lockouts, which are often a concern when using Selenium.

In [2]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager, ChromeType
import webdriver_manager

In [3]:
apt_data = pd.read_csv('F:/Rodrigo/Antiguo Escritorio/Rodrigo/Cursos y Proyectos/Data Projects/Web Scrapping/Resources/9-7-2023 (1).csv', header=0)
apt_columns = apt_data.columns.tolist()
apt_data.columns = apt_columns[1:] + ['XX']

print(apt_data.columns)
print(len(apt_data))
apt_data.head()

Index(['City', 'State', 'Zip', 'Auction Date', 'Sale Type', 'Auction Status',
       'Case Number', 'Bedrooms', 'Baths', 'Lot Size', 'Subdivision',
       'Property Type', 'Parcel Number', 'Owner 1 Name', 'Owner 1 First Name',
       'Owner 1 Last Name', 'Owner 2 Name', 'Owner 2 First Name',
       'Owner 2 Last Name', 'POV', 'County Market Value',
       'Final Judgement Amount', 'Plaintiff Max Bid', 'Plaintiff',
       'Defendants', 'Opening Bid', 'XX'],
      dtype='object')
260


,City,State,Zip,Auction Date,Sale Type,Auction Status,Case Number,Bedrooms,Baths,Lot Size,...,Owner 2 First Name,Owner 2 Last Name,POV,County Market Value,Final Judgement Amount,Plaintiff Max Bid,Plaintiff,Defendants,Opening Bid,XX
17390 Sw 21st St,Miramar,FL,33029,2023-09-08,FORECLOSURE,Upcoming,COWE-18-006360,4.0,2.5,5275.0,...,NaN,NaN,692608.0,469220,31834.28,31834.28,SILVERLAKES COMMUNITY ASSOCIATION INC,"ALFARO, LUIS, DOE, JOHN, DOE, JANE, TAYLOR, ST...",NaN,NaN
17461 Sw 33rd St,Miramar,FL,33029,2023-09-08,FORECLOSURE,Upcoming,COWE-20-021648,5.0,3.0,6024.0,...,NaN,NaN,788690.0,487290,16562.24,16562.24,SILVERLAKES COMMUNITY ASSOCIATION INC.,"DORITY , DIANE, DOE, JOHN, DOE, JANE, DSOUZA, ...",NaN,NaN
3481 Nw 122nd Ave,Sunrise,FL,33323,2023-09-08,FORECLOSURE,Upcoming,CONO-21-009761,3.0,2.0,6600.0,...,NaN,NaN,454429.0,321040,15899.95,15899.95,SAWGRASS ESTATES NORTH HOMEOWNERS ASSOCIATION INC,"JOHNSON, MICHAEL",NaN,NaN
1003 Springdale Cir,Palm Springs,FL,33461,2023-09-11,FORECLOSURE,Upcoming,502016CA003904XXXXMB,3.0,2.5,1559.0,...,NaN,NaN,131031.0,213372,361386.72,361386.72,CITIGROUP MORTGAGE LOAN TRUST INC ASSETBACKED ...,NICOLE CENAT; SEMIRANUS CENAT; UNKNOWN SPOUSE ...,NaN,NaN
128 Monterey Way,Royal Palm Beach,FL,33411,2023-09-11,FORECLOSURE,Upcoming,502022CA004022XXXXMB,5.0,3.0,13425.0,...,CAROLYN,ROSE,607380.0,537010,629778.72,629778.72,DEUTSCHE BANK NATIONAL TRUST COMPANY AS INDENT...,MICHAEL ROSE; CAROLYN ROSE; SARATOGA PINES HOM...,NaN,NaN


In [4]:
address_list = apt_data.index.tolist()

### Exploring Redfin:
Ultimately, I need to usee request and Beautiful Soup to extract the information needed.
I'm looking for the sold information as well as the color of the button that indicates the status of the appartment.

##### Exmaple:
https://www.redfin.com/FL/Miramar/17390-SW-21st-St-33029/home/41861501

In [9]:
url = "https://www.redfin.com/FL/Miramar/17390-SW-21st-St-33029/home/41861501"
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    # Now you can use BeautifulSoup to extract data from the parsed HTML.
else:
    print("Failed to retrieve the page:", response.status_code)

print(soup)

<!DOCTYPE html>
<html lang="en"><head>
<script charset="UTF-8" data-domain-script="7e5bc3d6-ef20-4760-aa0d-c8df4649fae2" src="https://cdn.cookielaw.org/scripttemplates/otSDKStub.js" type="text/javascript"></script>
<script ccpa-opt-out-geo="ca" ccpa-opt-out-ids="C0004" ccpa-opt-out-lspa="false" charset="UTF-8" src="https://cdn.cookielaw.org/opt-out/otCCPAiab.js" type="text/javascript"></script>
<!-- Server: phantom-306 --><!-- Time generated: Sat Sep 16 2023 09:14:08 GMT-0700 (Pacific Daylight Time) --><script>(function(a){window.__reactServerOnClickHandler=function(i){(a[i]=a[i]||[]).push(new Date)}})(window.__reactServerUnhandledEvents={})</script><title>17390 SW 21st St, Miramar, FL 33029 | Redfin</title><link data-href="CopBpWidgets~493df0b3.7eff92e178335729f09b.styles.css" data-react-server-css="" href="https://ssl.cdn-redfin.com/vLATEST/corvstatic/customer-pages/CopBpWidgets~493df0b3.7eff92e178335729f09b.styles.css" media="" rel="stylesheet" type="text/css"/><link data-href="bp-l

In [6]:
pattern1 = r"sold for (\$[\d,]+) on ((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{1,2}, \d{4})"
text_elements = soup.find_all(string=True)

for text_element in text_elements:
    match = re.search(pattern1, text_element)
    if match:
        sold_info = match.group(1)
        sale_date = match.group(2)
        print("Sold Information:", sold_info)
        print("Sale Date:", sale_date)

pattern = r'ListingStatusBannerSection--statusDot" style="background-color:(#[0-9a-fA-F]+)"'
text = str(soup)  # Convert the BeautifulSoup object to a string

match = re.search(pattern, text)

if match:
    color_code = match.group(1)
    print("Color Code:", color_code)
else:
    print("Color code not found in the HTML content.")

Sold Information: $305,000
Sale Date: Jun 7, 2006
Color Code: #0b68bd


In [8]:
url = "https://www.redfin.com/FL/Miramar/17390-SW-21st-St-33029/home/41861501"
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    # Now you can use BeautifulSoup to extract data from the parsed HTML.
else:
    print("Failed to retrieve the page:", response.status_code)

pattern1 = r"sold for (\$[\d,]+) on ((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{1,2}, \d{4})"
text_elements = soup.find_all(string=True)

scrap_data = []

for text_element in text_elements:
    match = re.search(pattern1, text_element)
    if match:
        sold_info = match.group(1)
        sale_date = match.group(2)

        # Extract color_code (moved this part inside the loop)
        pattern = r'ListingStatusBannerSection--statusDot" style="background-color:(#[0-9a-fA-F]+)"'
        text = str(soup)  # Convert the BeautifulSoup object to a string
        match_color = re.search(pattern, text)
        if match_color:
            color_code = match_color.group(1)
        else:
            color_code = "Not Found"

        scrap_data.append([sold_info, sale_date, color_code])

scrap_df = pd.DataFrame(scrap_data, columns=['sold_info', 'sale_date', 'color_code'])

scrap_df.head()

,sold_info,sale_date,color_code
0,"$305,000","Jun 7, 2006",#0b68bd


### Test 1 - Using Selenium
What I need now is to find the URL of the apartment of interest. Navigating through the Redfin home page would be a potential solution.
Selenium and Webdrive are powerful and interesting tools, but they need very fine tuning and I keep getting lockout after a few tries, which is very common.

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager, ChromeType
import webdriver_manager  # Import the webdriver_manager module

options = Options()
options.add_experimental_option("detach", True)

# Specify a custom directory for Chromedriver cache
webdriver_manager.cache_root = "F:/Rodrigo/Antiguo Escritorio/Rodrigo/Cursos y Proyectos/Data Projects/Web Scrapping/Cache"

# Initialize the WebDriver with the specified cache directory
driver_path = ChromeDriverManager(chrome_type=ChromeType.GOOGLE).install()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get("https://www.redfin.com/")
driver.implicitly_wait(10)

# Use the By module to locate the search bar
search_bar = driver.find_element(By.ID, "search-box-input")

search_query = "17390 Sw 21st St"
search_bar.send_keys(search_query)

# Use the By module to locate the search button
search_button = driver.find_element(By.CSS_SELECTOR, "button[data-rf-test-name='searchButton']")
search_button.click()

driver.implicitly_wait(10)
current_url = driver.current_url

print("Current URL:", current_url)

driver.quit()


PermissionError: [WinError 5] Acceso denegado: 'C:\\Users\\MAQ\\.wdm\\drivers\\chromedriver\\win64\\116.0.5845.179\\chromedriver-win32\\chromedriver.exe' -> 'C:\\Users\\MAQ\\.wdm\\drivers\\chromedriver\\win64\\116.0.5845.179\\chromedriver.exe'

In [31]:
options = Options()
options.add_experimental_option("detach", True)
driver_path = ChromeDriverManager(chrome_type=ChromeType.GOOGLE).install()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)

driver.get("https://www.redfin.com/")
driver.implicitly_wait(10)

# Use the By module to locate the search bar
search_bar = driver.find_element(By.ID, "search-box-input")

search_query = "17390 Sw 21st St"
search_bar.send_keys(search_query)

# Use the By module to locate the search button
search_button = driver.find_element(By.CSS_SELECTOR, "button[data-rf-test-name='searchButton']")
search_button.click()

driver.implicitly_wait(10)
current_url = driver.current_url

print("Current URL:", current_url)

driver.quit()


PermissionError: [WinError 5] Acceso denegado: 'C:\\Users\\MAQ\\.wdm\\drivers\\chromedriver\\win64\\116.0.5845.179\\chromedriver-win32\\chromedriver.exe' -> 'C:\\Users\\MAQ\\.wdm\\drivers\\chromedriver\\win64\\116.0.5845.179\\chromedriver.exe'

In [ ]:
driver = webdriver.Chrome()  # You'll need to install Chrome WebDriver: https://chromedriver.chromium.org/

for address in address_list:
    # Generate the search URL for the address
    search_url = f"https://www.redfin.com/search?query={address.replace(' ', '+')}"

    # Send a request to the search URL and parse the page with BeautifulSoup
    driver.get(search_url)
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Activate the search button to access the property page
    search_button = driver.find_element_by_css_selector("button[data-rf-test-name='searchButton']")
    search_button.click()

    # Extract the data using the provided code
    pattern1 = r"sold for (\$[\d,]+) on ((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{1,2}, \d{4})"
    text_elements = soup.find_all(text=True)

    scrap_data = []

    for text_element in text_elements:
        match = re.search(pattern1, text_element)
        if match:
            sold_info = match.group(1)
            sale_date = match.group(2)

            # Extract color_code (moved this part inside the loop)
            pattern = r'ListingStatusBannerSection--statusDot" style="background-color:(#[0-9a-fA-F]+)"'
            text = str(soup)  # Convert the BeautifulSoup object to a string
            match_color = re.search(pattern, text)
            if match_color:
                color_code = match_color.group(1)
            else:
                color_code = "Not Found"

            scrap_data.append([sold_info, sale_date, color_code])

    # Create a DataFrame from the extracted data
    scrap_df = pd.DataFrame(scrap_data, columns=['sold_info', 'sale_date', 'color_code'])

    # Add the address as the index
    scrap_df.index = [address] * len(scrap_df)

    # Append the data to the result DataFrame
    result_df = pd.concat([result_df, scrap_df])

# Close the webdriver
driver.quit()

# Print the final DataFrame
print(result_df)

### Test 2 - Using Google Search Engine to collect the URLs
Given the recurring issue of lockouts while using Selenium, it became evident that this approach was not a practical solution. Instead, I decided to use alternative method involving Google Search.
The process involves Googling each address from the dataset and gathering the URLs from the search results, provided they lead to a Redfin page. This methodology can be applied to other web sites, provided we can extract the URL and the desired data from the HTML code using Requests and Beautiful Soup.

##### Example
https://www.google.com/search?q=17390 Sw 21st St

In [5]:
address_list[0:5]

['17390 Sw 21st St',
 '17461 Sw 33rd St',
 '3481 Nw 122nd Ave',
 '1003 Springdale Cir',
 '128 Monterey Way']

In [72]:
address = ['1003 Springdale Cir']
google_url = f"https://www.google.com/search?q={address}"
print(google_url)

google_search = requests.get(google_url)

if google_search.status_code == 200:
    google_soup = BeautifulSoup(google_search.content, 'html.parser')
else:
    print("Failed to retrieber google search")
    
url_pattern = re.compile(r'https://www\.redfin\.com/[^\s&]+')
google_soup_str = str(google_soup)
url_match = url_pattern.findall(google_soup_str)[0]
print(url_match)

response = requests.get(url_match)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
else:
    print("Failed to retrieve the page:", response.status_code)
    
# Last Sold Info
pattern2 = r"sold for (\$[\d,]+) on ((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{1,2}, \d{4})"
text_elements = soup.find_all(string=True)

scrap_data = []

for text_element in text_elements:
    match = re.search(pattern2, text_element)
    if match:
        sold_info = match.group(1)
        sale_date = match.group(2)

        # Extract the color code
        pattern = r'ListingStatusBannerSection--statusDot" style="background-color:(#[0-9a-fA-F]+)"'
        text = str(soup)  # Convert the BeautifulSoup object to a string
        match_color = re.search(pattern, text)
        if match_color:
            color_code = match_color.group(1)
        else:
            color_code = "Not Found"

        scrap_data.append([sold_info, sale_date, color_code])

scrap_df = pd.DataFrame(scrap_data, columns=['sold_info', 'sale_date', 'color_code'], index = address)

scrap_df.head()

https://www.google.com/search?q=['1003 Springdale Cir']
https://www.redfin.com/FL/Palm-Springs/1003-Springdale-Cir-33461/home/42484227


,sold_info,sale_date,color_code
1003 Springdale Cir,NaN,NaN,NaN


In [54]:
address = ['17461 Sw 33rd St']
google_url = f"https://www.google.com/search?q={address}"
google_search = requests.get(google_url)
google_soup = BeautifulSoup(google_search.content, 'html.parser')

url_pattern = re.compile(r'https://www\.redfin\.com/[^\s&]+')
google_soup_str = str(google_soup)
url_match = url_pattern.findall(google_soup_str)[0]

response = requests.get(url_match)
soup = BeautifulSoup(response.content, "html.parser")
    
# Current estimated price
price_pattern = re.compile(r'<div class="price">(\$[0-9,]+)</div>')
match = price_pattern.search(str(soup))
estimated_price = match.group(1)

# Last Sold Info
pattern2 = r"sold for (\$[\d,]+) on ((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{1,2}, \d{4})"
text_elements = soup.find_all(string=True)

scrap_data = []

for text_element in text_elements:
    match = re.search(pattern2, text_element)
    if match:
        sold_info = match.group(1)
        sale_date = match.group(2)

        # Extract color_code
        pattern = r'ListingStatusBannerSection--statusDot" style="background-color:(#[0-9a-fA-F]+)"'
        text = str(soup)
        match_color = re.search(pattern, text)
        color_code = match_color.group(1)
        scrap_data.append([estimated_price,sold_info, sale_date, color_code])

scrap_df = pd.DataFrame(scrap_data, columns=['estimated_price','sold_info', 'sale_date', 'color_code'], index = address)

scrap_df.head()

,estimated_price,sold_info,sale_date,color_code
17461 Sw 33rd St,"$756,781","$435,000","Aug 8, 2016",#0b68bd


#### Futher tests
Once I identified the distinct colors associated with the buttons and their respective statuses, I had to create a dictionary and implement various actions to extract the desired data uniquely for each market status.

In [10]:
color_status_dict = {'pending': "#ec3f27", 'off_market': "#0b68bd", 'for_sale': "#067741"}
color_status_dict

{'pending': '#ec3f27', 'off_market': '#0b68bd', 'for_sale': '#067741'}

In [11]:
address = ['17390 Sw 21st St']
google_url = f"https://www.google.com/search?q={address}"
google_search = requests.get(google_url)
google_soup = BeautifulSoup(google_search.content, 'html.parser')

url_pattern = re.compile(r'https://www\.redfin\.com/[^\s&]+')
google_soup_str = str(google_soup)
url_match = url_pattern.findall(google_soup_str)[0]

response = requests.get(url_match)
soup = BeautifulSoup(response.content, "html.parser")

# Extract color_code
pattern_color = r'ListingStatusBannerSection--statusDot" style="background-color:(#[0-9a-fA-F]+)"'
text = str(soup)
match_color = re.search(pattern_color, text)
color_code = match_color.group(1)

# Initialize data to None
estimated_price = None
sold_info = None
sale_date = None

# Determine which data to extract based on the color_code
if color_code == color_status_dict['pending']:
    # If the color is pending, extract estimated_price only
    price_pattern = re.compile(r'<div class="price">(\$[0-9,]+)</div>')
    match = price_pattern.search(str(soup))
    if match:
        estimated_price = match.group(1)
    else:
        # If no match, find any $[0-9,]{5:} pattern and extract that price
        price_pattern_alt = re.compile(r'(\$[0-9,]{5,})')
        match_alt = price_pattern_alt.search(str(soup))
        if match_alt:
            estimated_price = match_alt.group(1)
elif color_code == color_status_dict['off_market']:
    # If the color is off_market, extract estimated_price, sold_info, and sale_date
    price_pattern = re.compile(r'<div class="price">(\$[0-9,]+)</div>')
    match = price_pattern.search(str(soup))
    if match:
        estimated_price = match.group(1)
    else:
        # If no match, find any $[0-9,]{5:} pattern and extract that price
        price_pattern_alt = re.compile(r'(\$[0-9,]{5,})')
        match_alt = price_pattern_alt.search(str(soup))
        if match_alt:
            estimated_price = match_alt.group(1)

    # Last Sold Info
    pattern2 = r"sold for (\$[\d,]+) on ((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{1,2}, \d{4})"
    text_elements = soup.find_all(string=True)

    for text_element in text_elements:
        match = re.search(pattern2, text_element)
        if match:
            sold_info = match.group(1)
            sale_date = match.group(2)

# Create a DataFrame
scrap_data = [[estimated_price, sold_info, sale_date, color_code]]
scrap_df = pd.DataFrame(scrap_data, columns=['estimated_price', 'sold_info', 'sale_date', 'color_code'], index=address)

# Display the DataFrame
print(scrap_df)

                 estimated_price sold_info    sale_date color_code
17390 Sw 21st St        $618,526  $305,000  Jun 7, 2006    #0b68bd


In [35]:
scrap_df = pd.DataFrame(columns=['estimated_price','sold_info', 'sale_date', 'color_code'], index = address_list)
scrap_df.head()

for address in address_list:
    google_url = f"https://www.google.com/search?q={address}"
    google_search = requests.get(google_url)
    google_soup = BeautifulSoup(google_search.content, 'html.parser')

    url_pattern = re.compile(r'https://www\.redfin\.com/[^\s&]+')
    google_soup_str = str(google_soup)
    url_match = url_pattern.findall(google_soup_str)[0]

    response = requests.get(url_match)
    soup = BeautifulSoup(response.content, "html.parser")

    # Extract color_code
    pattern_color = r'ListingStatusBannerSection--statusDot" style="background-color:(#[0-9a-fA-F]+)"'
    text = str(soup)
    match_color = re.search(pattern_color, text)
    color_code = match_color.group(1)

    # Initialize data to None
    estimated_price = None
    sold_info = None
    sale_date = None

    # Determine which data to extract based on the color_code
    if color_code == color_status_dict['pending'] or color_code == color_status_dict['for_sale']:
        # If the color is pending, extract estimated_price only
        price_pattern = re.compile(r'<div class="price">(\$[0-9,]+)</div>')
        match = price_pattern.search(str(soup))
        if match:
            estimated_price = match.group(1)
        else:
            # If no match, find any $[0-9,]{5:} pattern and extract that price
            price_pattern_alt = re.compile(r'(\$[0-9,]{5,})')
            match_alt = price_pattern_alt.search(str(soup))
            if match_alt:
                estimated_price = match_alt.group(1)
    elif color_code == color_status_dict['off_market']:
        # If the color is off_market, extract estimated_price, sold_info, and sale_date
        price_pattern = re.compile(r'<div class="price">(\$[0-9,]+)</div>')
        match = price_pattern.search(str(soup))
        if match:
            estimated_price = match.group(1)
        else:
            # If no match, find any $[0-9,]{5:} pattern and extract that price
            price_pattern_alt = re.compile(r'(\$[0-9,]{5,})')
            match_alt = price_pattern_alt.search(str(soup))
            if match_alt:
                estimated_price = match_alt.group(1)

        # Last Sold Info
        pattern2 = r"sold for (\$[\d,]+) on ((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{1,2}, \d{4})"
        text_elements = soup.find_all(string=True)

        for text_element in text_elements:
            match = re.search(pattern2, text_element)
            if match:
                sold_info = match.group(1)
                sale_date = match.group(2)
        
    # Dataframe for the current address
    scrap_data = []
    scrap_data = [[estimated_price, sold_info, sale_date, color_code]]
    current_df = pd.DataFrame(scrap_data, columns=['estimated_price','sold_info', 'sale_date', 'color_code'])
    
    # Append current_df to scrap_df
    scrap_df.loc[address] = current_df.iloc[0]

# Display the result DataFrame
print(scrap_df)
    

ValueError: shape mismatch: value array of shape (2,) could not be broadcast to indexing result of shape (2,4)

In [85]:
scrap_df.head(20)

,estimated_price,sold_info,sale_date,color_code
17390 Sw 21st St,"$618,526","$305,000","Jun 7, 2006",#0b68bd
17461 Sw 33rd St,"$756,781","$435,000","Aug 8, 2016",#0b68bd
3481 Nw 122nd Ave,"$511,501","$226,000","Apr 14, 2014",#0b68bd
1003 Springdale Cir,"$135,000",None,None,#ec3f27
128 Monterey Way,"$637,376",None,None,#ec3f27
12861 62nd Ln N,None,None,None,#067741
2205 Ne 4th St,"$306,075","$25,300","Jan 1, 1978",#0b68bd
404 Woodview Cir,"$575,891",None,None,#ec3f27
4403 Nw 2nd Ave,"$529,794",None,None,#ec3f27
7328 Nautica Way,"$492,940","$137,000","Nov 30, 2000",#0b68bd


### Test 3: Exploring other statuses
I began to observe positive outcomes with this approach. I simply had to fine-tune the code to handle various scenarios, such as cases where the apartment wasn't listed on Redfin or when there were inconsistencies in the HTML code on the Redfin web page.

In [8]:
address = ['5828 Strawberry Lakes Cir'] ## For sale
google_url = f"https://www.google.com/search?q={address}"
google_search = requests.get(google_url)
google_soup = BeautifulSoup(google_search.content, 'html.parser')

url_pattern = re.compile(r'https://www\.redfin\.com/[^\s&]+')
google_soup_str = str(google_soup)
url_match = url_pattern.findall(google_soup_str)[0]

response = requests.get(url_match)
soup = BeautifulSoup(response.content, "html.parser")
print(url_match)

https://www.redfin.com/FL/Lake-Worth/5828-Strawberry-Lakes-Cir-33463/home/42178320


In [19]:
f_path = f"F:/Rodrigo/Antiguo Escritorio/Rodrigo/Cursos y Proyectos/Data Projects/Web Scrapping/Resources/b_soup_{address[0]}.txt"
with open(f_path, "w", encoding="utf-8") as file:
    file.write(str(soup))

In [39]:
address = ['5828 Strawberry Lakes Cir']
google_url = f"https://www.google.com/search?q={address}"
google_search = requests.get(google_url)
google_soup = BeautifulSoup(google_search.content, 'html.parser')

url_pattern = re.compile(r'https://www\.redfin\.com/[^\s&]+')
google_soup_str = str(google_soup)
url_match = url_pattern.findall(google_soup_str)[0]

response = requests.get(url_match)
soup = BeautifulSoup(response.content, "html.parser")

# Extract color_code
pattern_color = r'ListingStatusBannerSection--statusDot" style="background-color:(#[0-9a-fA-F]+)"'
text = str(soup)
match_color = re.search(pattern_color, text)
color_code = match_color.group(1)

# Initialize data to None
estimated_price = None
sold_info = None
sale_date = None

# Determine which data to extract based on the color_code
if color_code in [color_status_dict['pending'], color_status_dict['for_sale']]:
    # If the color is pending, extract estimated_price only
    price_pattern = re.compile(r'<div class="price">(\$[0-9,]+)</div>')
    match = price_pattern.search(str(soup))
    if match:
        estimated_price = match.group(1)
    else:
        # If no match, find any $[0-9,]{5:} pattern and extract that price
        price_pattern_alt = re.compile(r'(\$[0-9,]{5,})')
        match_alt = price_pattern_alt.search(str(soup))
        if match_alt:
            estimated_price = match_alt.group(1)
elif color_code == color_status_dict['off_market']:
    # If the color is off_market, extract estimated_price, sold_info, and sale_date
    price_pattern = re.compile(r'<div class="price">(\$[0-9,]+)</div>')
    match = price_pattern.search(str(soup))
    if match:
        estimated_price = match.group(1)
    else:
        # If no match, find any $[0-9,]{5:} pattern and extract that price
        price_pattern_alt = re.compile(r'(\$[0-9,]{5,})')
        match_alt = price_pattern_alt.search(str(soup))
        if match_alt:
            estimated_price = match_alt.group(1)

    # Last Sold Info
    pattern2 = r"sold for (\$[\d,]+) on ((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{1,2}, \d{4})"
    text_elements = soup.find_all(string=True)

    for text_element in text_elements:
        match = re.search(pattern2, text_element)
        if match:
            sold_info = match.group(1)
            sale_date = match.group(2)

# Create a DataFrame
scrap_data = [[estimated_price, sold_info, sale_date, color_code]]
scrap_df = pd.DataFrame(scrap_data, columns=['estimated_price', 'sold_info', 'sale_date', 'color_code'], index=address)

# Display the DataFrame
print(scrap_df)
print(type(scrap_df))
print(scrap_df.shape)
print(type(scrap_df['estimated_price'][0]))

                          estimated_price sold_info sale_date color_code
5828 Strawberry Lakes Cir        $433,841      None      None    #067741
<class 'pandas.core.frame.DataFrame'>
(1, 4)
<class 'str'>


C:\Users\MAQ\AppData\Local\Temp\ipykernel_27496\1089542399.py:68: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(type(scrap_df['estimated_price'][0]))


In [41]:
# Initialize an empty list to collect data for all addresses
data_list = []

for address in address_list:
    google_url = f"https://www.google.com/search?q={address}"
    google_search = requests.get(google_url)
    google_soup = BeautifulSoup(google_search.content, 'html.parser')

    url_pattern = re.compile(r'https://www\.redfin\.com/[^\s&]+')
    google_soup_str = str(google_soup)
    url_match = url_pattern.findall(google_soup_str)[0]

    response = requests.get(url_match)
    soup = BeautifulSoup(response.content, "html.parser")

    # Extract color_code
    pattern_color = r'ListingStatusBannerSection--statusDot" style="background-color:(#[0-9a-fA-F]+)"'
    text = str(soup)
    match_color = re.search(pattern_color, text)

    if match_color:
        color_code = match_color.group(1)
    else:
        color_code = None

    # Initialize data to None
    estimated_price = None
    sold_info = None
    sale_date = None

    # Determine which data to extract based on the color_code
    if color_code in [color_status_dict['pending'], color_status_dict['for_sale']]:
        # If the color is pending or for_sale, extract estimated_price only
        price_pattern = re.compile(r'<div class="price">(\$[0-9,]+)</div>')
        match = price_pattern.search(str(soup))
        if match:
            estimated_price = match.group(1)
        else:
            # If no match, find any $[0-9,]{5:} pattern and extract that price
            price_pattern_alt = re.compile(r'(\$[0-9,]{5,})')
            match_alt = price_pattern_alt.search(str(soup))
            if match_alt:
                estimated_price = match_alt.group(1)
    elif color_code == color_status_dict['off_market']:
        # If the color is off_market, extract estimated_price, sold_info, and sale_date
        price_pattern = re.compile(r'<div class="price">(\$[0-9,]+)</div>')
        match = price_pattern.search(str(soup))
        if match:
            estimated_price = match.group(1)
        else:
            # If no match, find any $[0-9,]{5:} pattern and extract that price
            price_pattern_alt = re.compile(r'(\$[0-9,]{5,})')
            match_alt = price_pattern_alt.search(str(soup))
            if match_alt:
                estimated_price = match_alt.group(1)

        # Last Sold Info
        pattern2 = r"sold for (\$[\d,]+) on ((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{1,2}, \d{4})"
        text_elements = soup.find_all(string=True)

        for text_element in text_elements:
            match = re.search(pattern2, text_element)
            if match:
                sold_info = match.group(1)
                sale_date = match.group(2)
    
    interm_list = [address, estimated_price, sold_info, sale_date, color_code]

    # Append data for the current address to the data_list
    data_list.append([address, estimated_price, sold_info, sale_date, color_code])
    print(address)
    print(interm_list)

# Create a DataFrame from the collected data
print(data_list)
scrap_df = pd.DataFrame(data_list, columns=['address', 'estimated_price', 'sold_info', 'sale_date', 'color_code'])
scrap_df.set_index('address', inplace=True)

# Display the DataFrame
print(scrap_df)

17390 Sw 21st St
['17390 Sw 21st St', '$618,526', '$305,000', 'Jun 7, 2006', '#0b68bd']
17461 Sw 33rd St
['17461 Sw 33rd St', '$756,781', '$435,000', 'Aug 8, 2016', '#0b68bd']
3481 Nw 122nd Ave
['3481 Nw 122nd Ave', '$511,501', '$226,000', 'Apr 14, 2014', '#0b68bd']
1003 Springdale Cir
['1003 Springdale Cir', '$135,000', None, None, '#ec3f27']
128 Monterey Way
['128 Monterey Way', '$637,734', None, None, '#ec3f27']
12861 62nd Ln N
['12861 62nd Ln N', '$389,447', None, None, '#ec3f27']
2205 Ne 4th St
['2205 Ne 4th St', '$306,075', '$25,300', 'Jan 1, 1978', '#0b68bd']
404 Woodview Cir
['404 Woodview Cir', '$658,311', '$357,000', 'Oct 21, 2019', '#0b68bd']
4403 Nw 2nd Ave
['4403 Nw 2nd Ave', '$527,580', None, None, '#ec3f27']
7328 Nautica Way
['7328 Nautica Way', '$492,940', '$137,000', 'Nov 30, 2000', '#0b68bd']
1221 Ne 14th Ave
['1221 Ne 14th Ave', '$1,088,574', '$450,000', 'Oct 3, 2012', '#0b68bd']
12875 Sw 50th Ct
['12875 Sw 50th Ct', '$357,871', None, None, '#ec3f27']
3180 Carol Ave


IndexError: list index out of range

In [42]:
address_list

['17390 Sw 21st St',
 '17461 Sw 33rd St',
 '3481 Nw 122nd Ave',
 '1003 Springdale Cir',
 '128 Monterey Way',
 '12861 62nd Ln N',
 '2205 Ne 4th St',
 '404 Woodview Cir',
 '4403 Nw 2nd Ave',
 '7328 Nautica Way',
 '1221 Ne 14th Ave',
 '12875 Sw 50th Ct',
 '3180 Carol Ave',
 '3541 Nw 37th Ave',
 '4357 Sw 125th Ln',
 '5239 Nw 96th Ave',
 '5828 Strawberry Lakes Cir',
 '8630 Nw 25th St',
 '11945 Sw 16th Ct',
 '16701 84th Ct N',
 '1741 Nw 106th Ave',
 '18940 Sw 31st Ct',
 '2300 Nw 38th Ter',
 '3820 Ne 29th Ave',
 '543 S Crescent Dr',
 '571 Nw 41st St',
 '6602 Boulevard Of Champions',
 '7771 Nw 46th St',
 '922 S Northlake Dr',
 '124 Allen Rd',
 '13300 Sw 20th St',
 '427 Maddock St',
 '5568 Nw 90th Ter',
 '5801 Sw 33rd Ave',
 '6424 96th Pl S',
 '6457 Nw 99th Ave',
 '725 Nw 2nd St',
 '955 Nw 197th Ave',
 '4101 W Silverado Cir',
 '6844 Nw 70th Pl',
 '1390 W 33rd St',
 '16394 E Cornwall Dr',
 '1709 22nd Ave N',
 '18681 42nd Rd N',
 '5954 Las Colinas Cir',
 '6074 Rossmoor Lakes Ct',
 '7270 Spinnaker

In [43]:
address = ['484 S Dixie Hwy W'] ## Not available in Redfin
google_url = f"https://www.google.com/search?q={address}"
google_search = requests.get(google_url)
google_soup = BeautifulSoup(google_search.content, 'html.parser')

url_pattern = re.compile(r'https://www\.redfin\.com/[^\s&]+')
google_soup_str = str(google_soup)
url_match = url_pattern.findall(google_soup_str)[0]

response = requests.get(url_match)
soup = BeautifulSoup(response.content, "html.parser")
print(url_match)

f_path = f"F:/Rodrigo/Antiguo Escritorio/Rodrigo/Cursos y Proyectos/Data Projects/Web Scrapping/Resources/b_soup_{address[0]}.txt"
with open(f_path, "w", encoding="utf-8") as file:
    file.write(str(soup))

IndexError: list index out of range

In [45]:
data_list = []

for address in address_list:
    google_url = f"https://www.google.com/search?q={address}"
    google_search = requests.get(google_url)
    google_soup = BeautifulSoup(google_search.content, 'html.parser')

    url_pattern = re.compile(r'https://www\.redfin\.com/[^\s&]+')
    google_soup_str = str(google_soup)
    url_match = url_pattern.findall(google_soup_str)

    if url_match:
        url_match = url_match[0]
        response = requests.get(url_match)
        soup = BeautifulSoup(response.content, "html.parser")

        # Extract color_code
        pattern_color = r'ListingStatusBannerSection--statusDot" style="background-color:(#[0-9a-fA-F]+)"'
        text = str(soup)
        match_color = re.search(pattern_color, text)

        if match_color:
            color_code = match_color.group(1)
        else:
            color_code = None

        # Initialize data to None
        estimated_price = None
        sold_info = None
        sale_date = None

        # Determine which data to extract based on the color_code
        if color_code in [color_status_dict['pending'], color_status_dict['for_sale']]:
            # If the color is pending or for_sale, extract estimated_price only
            price_pattern = re.compile(r'<div class="price">(\$[0-9,]+)</div>')
            match = price_pattern.search(str(soup))
            if match:
                estimated_price = match.group(1)
            else:
                # If no match, find any $[0-9,]{5:} pattern and extract that price
                price_pattern_alt = re.compile(r'(\$[0-9,]{5,})')
                match_alt = price_pattern_alt.search(str(soup))
                if match_alt:
                    estimated_price = match_alt.group(1)
        elif color_code == color_status_dict['off_market']:
            # If the color is off_market, extract estimated_price, sold_info, and sale_date
            price_pattern = re.compile(r'<div class="price">(\$[0-9,]+)</div>')
            match = price_pattern.search(str(soup))
            if match:
                estimated_price = match.group(1)
            else:
                # If no match, find any $[0-9,]{5:} pattern and extract that price
                price_pattern_alt = re.compile(r'(\$[0-9,]{5,})')
                match_alt = price_pattern_alt.search(str(soup))
                if match_alt:
                    estimated_price = match_alt.group(1)

            # Last Sold Info
            pattern2 = r"sold for (\$[\d,]+) on ((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{1,2}, \d{4})"
            text_elements = soup.find_all(string=True)

            for text_element in text_elements:
                match = re.search(pattern2, text_element)
                if match:
                    sold_info = match.group(1)
                    sale_date = match.group(2)
        
        interm_list = [address, estimated_price, sold_info, sale_date, color_code]

        # Append data for the current address to the data_list
        data_list.append(interm_list)
        print(address)
        print(interm_list)
    else:
        # Append None values for the address where url_match is not found
        data_list.append([address, None, None, None, None])

# Create a DataFrame from the collected data
scrap_df = pd.DataFrame(data_list, columns=['address', 'estimated_price', 'sold_info', 'sale_date', 'color_code'])
scrap_df.set_index('address', inplace=True)

17390 Sw 21st St
['17390 Sw 21st St', '$618,526', '$305,000', 'Jun 7, 2006', '#0b68bd']
17461 Sw 33rd St
['17461 Sw 33rd St', '$756,781', '$435,000', 'Aug 8, 2016', '#0b68bd']
3481 Nw 122nd Ave
['3481 Nw 122nd Ave', '$511,501', '$226,000', 'Apr 14, 2014', '#0b68bd']
1003 Springdale Cir
['1003 Springdale Cir', '$135,000', None, None, '#ec3f27']
128 Monterey Way
['128 Monterey Way', '$637,734', None, None, '#ec3f27']
12861 62nd Ln N
['12861 62nd Ln N', '$389,447', None, None, '#ec3f27']
2205 Ne 4th St
['2205 Ne 4th St', '$306,075', '$25,300', 'Jan 1, 1978', '#0b68bd']
404 Woodview Cir
['404 Woodview Cir', '$658,311', '$357,000', 'Oct 21, 2019', '#0b68bd']
4403 Nw 2nd Ave
['4403 Nw 2nd Ave', '$527,580', None, None, '#ec3f27']
7328 Nautica Way
['7328 Nautica Way', '$492,940', '$137,000', 'Nov 30, 2000', '#0b68bd']
1221 Ne 14th Ave
['1221 Ne 14th Ave', '$1,088,574', '$450,000', 'Oct 3, 2012', '#0b68bd']
12875 Sw 50th Ct
['12875 Sw 50th Ct', '$357,871', None, None, '#ec3f27']
3180 Carol Ave


In [46]:
print(len(address_list))

260


In [48]:
print(len(scrap_df))
scrap_df.head()

260


,estimated_price,sold_info,sale_date,color_code
address,,,,
17390 Sw 21st St,"$618,526","$305,000","Jun 7, 2006",#0b68bd
17461 Sw 33rd St,"$756,781","$435,000","Aug 8, 2016",#0b68bd
3481 Nw 122nd Ave,"$511,501","$226,000","Apr 14, 2014",#0b68bd
1003 Springdale Cir,"$135,000",None,None,#ec3f27
128 Monterey Way,"$637,734",None,None,#ec3f27


### Test 4 - Cleaning up the results
There are a few steps I need to take next in order to optimize the outcomes of the web scraping method.
1. Inclue city and state in the google search
2. Map de color_code with the status, using my dictionary
3. Make the final executable program

In [12]:
address_info = apt_data.iloc[:, :2]
address_info.head()

,City,State
17390 Sw 21st St,Miramar,FL
17461 Sw 33rd St,Miramar,FL
3481 Nw 122nd Ave,Sunrise,FL
1003 Springdale Cir,Palm Springs,FL
128 Monterey Way,Royal Palm Beach,FL


In [13]:
color_status_dict = {'pending': "#ec3f27", 'off_market': "#0b68bd", 'for_sale': "#067741"}
color_status_dict

{'pending': '#ec3f27', 'off_market': '#0b68bd', 'for_sale': '#067741'}

In [53]:
data_list = []

for address in address_list:
    city = address_info.loc[address, 'City']
    state = address_info.loc[address, 'State']
    
    google_url = f"https://www.google.com/search?q={address} {city} {state}"
    google_search = requests.get(google_url)
    google_soup = BeautifulSoup(google_search.content, 'html.parser')

    url_pattern = re.compile(r'https://www\.redfin\.com/[^\s&]+')
    google_soup_str = str(google_soup)
    url_match = url_pattern.findall(google_soup_str)

    if url_match:
        url_match = url_match[0]
        response = requests.get(url_match)
        soup = BeautifulSoup(response.content, "html.parser")

        # Extract color_code
        pattern_color = r'ListingStatusBannerSection--statusDot" style="background-color:(#[0-9a-fA-F]+)"'
        text = str(soup)
        match_color = re.search(pattern_color, text)

        if match_color:
            color_code = match_color.group(1)
        else:
            color_code = None

        # Initialize data to None
        estimated_price = None
        sold_info = None
        sale_date = None

        # Determine which data to extract based on the color_code
        if color_code in [color_status_dict['pending'], color_status_dict['for_sale']]:
            # If the color is pending or for_sale, extract estimated_price only
            price_pattern = re.compile(r'<div class="price">(\$[0-9,]+)</div>')
            match = price_pattern.search(str(soup))
            if match:
                estimated_price = match.group(1)
            else:
                # If no match, find any $[0-9,]{5:} pattern and extract that price
                price_pattern_alt = re.compile(r'(\$[0-9,]{5,})')
                match_alt = price_pattern_alt.search(str(soup))
                if match_alt:
                    estimated_price = match_alt.group(1)
        elif color_code == color_status_dict['off_market']:
            # If the color is off_market, extract estimated_price, sold_info, and sale_date
            price_pattern = re.compile(r'<div class="price">(\$[0-9,]+)</div>')
            match = price_pattern.search(str(soup))
            if match:
                estimated_price = match.group(1)
            else:
                # If no match, find any $[0-9,]{5:} pattern and extract that price
                price_pattern_alt = re.compile(r'(\$[0-9,]{5,})')
                match_alt = price_pattern_alt.search(str(soup))
                if match_alt:
                    estimated_price = match_alt.group(1)

            # Last Sold Info
            pattern2 = r"sold for (\$[\d,]+) on ((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{1,2}, \d{4})"
            text_elements = soup.find_all(string=True)

            for text_element in text_elements:
                match = re.search(pattern2, text_element)
                if match:
                    sold_info = match.group(1)
                    sale_date = match.group(2)
        
        interm_list = [address, estimated_price, sold_info, sale_date, color_code]

        # Append data for the current address to the data_list
        data_list.append(interm_list)
    else:
        # Append None values for the address where url_match is not found
        data_list.append([address, None, None, None, None])
        with open('log_data_list.txt', 'w') as file:
            for item in data_list:
                file.write(' '.join(map(str, item)) + '\n')

# Create a DataFrame from the collected data
scrap_df = pd.DataFrame(data_list, columns=['address', 'estimated_price', 'sold_info', 'sale_date', 'color_code'])
scrap_df.set_index('address', inplace=True)

status_color_dict = {v: k for k, v in color_status_dict.items()}
scrap_df['status'] = scrap_df['color_code'].map(status_color_dict)
scrap_df.drop(columns=['color_code'], inplace=True)
scrap_df_f.head()
scrap_df.to_excel('scrap_df.xlsx', index=True)